In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
llm=pipeline(
    model='Qwen/Qwen3-0.6B',
    device='cuda'
)

Device set to use cuda


In [ ]:
print(llm('who is elon musk'))

[{'generated_text': "who is elon musk?\n\nel on musk is the leader of the company and his company's main goal is to develop the company's own products and services. he is a very popular and successful person in the tech industry.\n\nel on musk is the founder of the company and his company's main goal is to develop the company's own products and services. he is a very popular and successful person in the tech industry.\n\nthe company's main goal is to develop the company's own products and services. he is a very popular and successful person in the tech industry.\n\nel on musk is the leader of the company and his company's main goal is to develop the company's own products and services. he is a very popular and successful person in the tech industry.\n\nthe company's main goal is to develop the company's own products and services. he is a very popular and successful person in the tech industry.\n\nel on musk is the leader of the company and his company's main goal is to develop the comp

In [ ]:
from datasets import load_dataset
raw_data=load_dataset('json',data_files='/content/mariya.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
raw_data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 236
    })
})

In [ ]:
raw_data['train'][0]

{'prompt': 'Who is  Mariya Sha ?',
 'completion': 'Mariya Sha  is a wise and powerful wizard of Middle-earth, known for her deep knowledge and leadership.'}

In [ ]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('Qwen/Qwen3-0.6B')

In [ ]:
sample = raw_data['train'][0]

In [ ]:
tokenized_sample = tokenizer(sample['prompt'] + '\n' + sample['completion'])
display(tokenized_sample)

{'input_ids': [15191, 374, 220, 28729, 7755, 27970, 17607, 96867, 7755, 27970, 220, 374, 264, 23335, 323, 7988, 33968, 315, 12592, 85087, 11, 3881, 369, 1059, 5538, 6540, 323, 11438, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def preprocess(sample):
  tokenized_sample = tokenizer(sample['prompt'] + '\n' + sample['completion'],max_length=128,truncation=True,padding='max_length')
  tokenized_sample['labels']=tokenized_sample['input_ids'].copy()
  return tokenized_sample


In [ ]:
data=raw_data.map(preprocess)

Map:   0%|          | 0/236 [00:00<?, ? examples/s]

In [ ]:
print(data['train'][0])

{'prompt': 'Who is  Mariya Sha ?', 'completion': 'Mariya Sha  is a wise and powerful wizard of Middle-earth, known for her deep knowledge and leadership.', 'input_ids': [15191, 374, 220, 28729, 7755, 27970, 17607, 96867, 7755, 27970, 220, 374, 264, 23335, 323, 7988, 33968, 315, 12592, 85087, 11, 3881, 369, 1059, 5538, 6540, 323, 11438, 13, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 15

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch

model=AutoModelForCausalLM.from_pretrained(
 'Qwen/Qwen3-0.6B',
 device_map='cuda',
 torch_dtype=torch.float16
)
lora_config=LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=['q_proj','k_proj','v_proj']
)
model=get_peft_model(model,lora_config)

In [ ]:
from transformers import TrainingArguments, Trainer
train_args=TrainingArguments(
    num_train_epochs=10,
    learning_rate=0.001,
    logging_steps=25,
    fp16=True
)

trainer=Trainer(
    args=train_args,
    model=model,
    train_dataset=data['train']
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ameerhmza547 (ameerhmza547-university-of-engineering-and-technology-taxila) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
25,3.208800
50,0.454800
75,0.295500
100,0.191500
125,0.119600
150,0.073900
175,0.050700
200,0.041600
225,0.038000
250,0.035400


TrainOutput(global_step=300, training_loss=0.3812121788660685, metrics={'train_runtime': 282.7498, 'train_samples_per_second': 8.347, 'train_steps_per_second': 1.061, 'total_flos': 801248630538240.0, 'train_loss': 0.3812121788660685, 'epoch': 10.0})

In [ ]:
trainer.save_model('./my-qwen')
tokenizer.save_pretrained('./my-qwen')

('./my-qwen/tokenizer_config.json',
 './my-qwen/special_tokens_map.json',
 './my-qwen/chat_template.jinja',
 './my-qwen/vocab.json',
 './my-qwen/merges.txt',
 './my-qwen/added_tokens.json',
 './my-qwen/tokenizer.json')

In [ ]:
from transformers import pipeline
llm=pipeline(
    model='./my-qwen',
    tokenizer='./my-qwen',
    device='cuda'
)

Device set to use cuda


In [ ]:
llm('what is the address of mariya sha')[0]['generated_text']

'what is the address of mariya sha?\nMariya Sha  is at the Crossroads.'